In [23]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
import os
import os, sklearn, pandas, numpy as np, random
from sklearn import svm
import skimage, skimage.io, skimage.filters
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
%matplotlib inline

In [2]:
# set cwd back to default
os.chdir('../')
os.getcwd()

'/Users/Tristan/Downloads/dog-breed-identification/src'

In [3]:
# custom scripts
import config # params, constants
import data, models # functions that mutate outr data
# from utils import utils, plot # custom functions, in local environment

 Dataset :: namedtuple(
    ['train' = ['img_name']
    , 'train_final' = ['img_name']
    , 'test' = ['img_name']
    , 'test_final' = ['img_name']
    , 'labels' = pandas.df('img_name','breed')
    , 'dict_index_to_label' = dict to convert label_index -> label_name
    , 'dict_label_to_index'= dict to convert label_name -> label_index
    


In [4]:
import data # src/data.py
dataset = data.init_dataset()

### Select the amount of classes that will be used

In [5]:
# pick the n classes with the most occuring instances
amt = 3
classes = data.top_classes(dataset.labels, amt)
classes

['scottish_deerhound', 'maltese_dog', 'afghan_hound']

### Variables

In [6]:
img_width, img_height = 256, 256
train_data_dir = "data/train"
validation_data_dir = "data/val"
# batch_size = 16
# epochs = 50

### ??????

In [7]:
name_list = []
n_per_class = []
tail = '.jpg'
for cls in classes:
    names = data.items_with_label(dataset.labels, cls)
    train_names = [f for f in names if (f + tail) in dataset.train]
    name_list.append(train_names)
    n_per_class.append(len(train_names))
    
n = min(n_per_class)
# (optional) reduce n to check whether the model can rember its input
reduced_n = 50
if n > reduced_n:    n = reduced_n
x = []
for ls in name_list:
    for name in ls[:n]:
        x.append(name)
random.shuffle(x)

### Load and convert images

In [8]:
x_train, y_train, n = data.extract_all(dataset, x)
y_test = y_train
y_train, y_test = data.labels_to_vectors(dataset, y_train, y_test)

input_shape = x_train.shape[1:] # = shape of an individual image (matrix)
output_length = (y_train[0]).shape[0] # = length of an individual label


('extract all data:', 150)


Initiate VG16 network and add extra layers

In [17]:
from keras.models import Model 
dropout = 0.10

# build the VGG16 network
model = applications.VGG16(weights='imagenet', include_top=False, input_shape = input_shape )
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

for layer in model.layers:
    layer.trainable = False
    
#Adding custom Layers 
x = model.output
x = Flatten()(x)
# x = Dense(1024, activation="relu")(x)
# x = Dropout(0.5)(x)
# x = Dense(1024, activation="relu")(x)
x = (Dense(128, activation='relu'))(x)
x = (Dense(16, activation='relu'))(x)
x = (Dropout(dropout))(x)
x = (Dense(output_length,activation='softmax'))(x)
predictions = Dense(output_length, activation="softmax")(x)


# creating the final model 
model_final = Model(input = model.input, output = predictions)

learning_rate = 0.001
# Adam, SGD
# sgd = Keras.optimizers.SGD(lr=0.01, clipnorm=1.)
optimizer = optimizers.Adam(lr=learning_rate)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=['accuracy',
    'mean_squared_error','categorical_crossentropy','top_k_categorical_accuracy'])

Model loaded.


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


data augmentation

In [19]:
#Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30)


### Train the model

In [20]:
batch_size = 16
# n epochs = n iterations over all the training data
epochs = 10

### With data augmentation

In [24]:
train_datagen.fit(x_train)

model_final.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) / batch_size, epochs=epochs, callbacks=[TensorBoard(log_dir=config.tmp_model_dir)])


Epoch 1/10
9/9 [==============================] - 92s 10s/step - loss: 4.5580 - acc: 0.2905 - mean_squared_error: 0.0093 - categorical_crossentropy: 4.5580 - top_k_categorical_accuracy: 0.3958
Epoch 2/10
9/9 [==============================] - 104s 12s/step - loss: 4.5021 - acc: 0.3232 - mean_squared_error: 0.0093 - categorical_crossentropy: 4.5021 - top_k_categorical_accuracy: 0.6628
Epoch 3/10
9/9 [==============================] - 98s 11s/step - loss: 4.4715 - acc: 0.2389 - mean_squared_error: 0.0093 - categorical_crossentropy: 4.4715 - top_k_categorical_accuracy: 0.8314
Epoch 4/10
9/9 [==============================] - 96s 11s/step - loss: 4.4366 - acc: 0.3021 - mean_squared_error: 0.0093 - categorical_crossentropy: 4.4366 - top_k_categorical_accuracy: 1.0000
Epoch 5/10
9/9 [==============================] - 108s 12s/step - loss: 4.4004 - acc: 0.3611 - mean_squared_error: 0.0093 - categorical_crossentropy: 4.4004 - top_k_categorical_accuracy: 1.0000
Epoch 6/10
9/9 [=================

### Without data augmentation

In [ ]:
from keras.callbacks import TensorBoard
model_final.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
          validation_split=1/6, callbacks=[TensorBoard(log_dir=config.tmp_model_dir)])

In [ ]:
model_final.summary()